In [1]:
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
import pandas as pd
from numpy import int16


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = '''
SELECT 
    (COALESCE(a.score_17a, 0) + COALESCE(a.score_21, 0) + COALESCE(a.score_21a, 0) + COALESCE(a.score_22, 0) + COALESCE(a.score_22a, 0) + COALESCE(a.score_23, 0)) AS vbg_score_total,
    p.patient_code AS dreams_code,
    dga.nbr_presence,
    a.*
FROM
    (SELECT 
        CASE
                WHEN d.a17 = '0' THEN 0
                WHEN d.a17 = '4' THEN 4
                WHEN d.a17 = '04' THEN 4
                WHEN d.a17 = '004' THEN 4
            END AS score_17a,
            CASE
                WHEN d.eske_ou_konn_bwe_alkol = 'oui' THEN 4
                WHEN d.eske_ou_konn_bwe_alkol = 'non' THEN 0
            END AS score_21,
            COALESCE(d.a21_ske_ou_bw_alkl_osnon_itilize_lt_dwg_10_14, a21_nan_ki_frenkans_ou_itilize_alkol) AS score_21a,
            CASE
                WHEN d.eske_ou_konn_itilize_dwog = 'oui' THEN 4
                WHEN d.eske_ou_konn_itilize_dwog = 'non' THEN 0
            END AS score_22,
            COALESCE(d.ske_ou_konn_itilize_dwg_10_14, ske_ou_konn_itilize_dwg_15_19) AS score_22a,
            CASE
                WHEN d.ou_konn_enkonsyan_apre_alcohol_ou_dwog = '5' THEN 3
                WHEN d.ou_konn_enkonsyan_apre_alcohol_ou_dwog = '0' THEN 0
            END AS score_23,
            d.*
    FROM
        caris_db.dreams_surveys_data d) a
        LEFT JOIN
    dream_member dm ON dm.case_id = a.case_id
        LEFT JOIN
    patient p ON p.id = dm.id_patient
        LEFT JOIN
    (SELECT 
        COUNT(DISTINCT dgs.topic) AS nbr_presence, id_patient
    FROM
        dream_group_attendance dga1
    LEFT JOIN dream_group_session dgs ON dga1.id_group_session = dgs.id
    WHERE
        dga1.value = 'P'
    GROUP BY dga1.id_patient) dga ON dga.id_patient = dm.id_patient
WHERE
    a.total > 14
ORDER BY (COALESCE(a.score_17a, 0) + COALESCE(a.score_21, 0) + COALESCE(a.score_21a, 0) + COALESCE(a.score_22, 0) + COALESCE(a.score_22a, 0) + COALESCE(a.score_23, 0)) DESC
'''

# close the pool of connection
engine.dispose()

vbg = pd.read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query
vbg.to_excel('./vbg.xlsx',index=False,na_rep="NULL")

In [2]:
vbg.vbg_score_total.count()

49642